**Project Question: Predict the hourly demand of taxi services in each zone in Manhattan NYC.**



In [0]:
# CHANGE TO YOUR EMAIL IN WORKSPACE
user = "zhaozhou.lyu@vanderbilt.edu"
# Now, should be able to run notebook fully

# Set Up

In [0]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, dayofweek, weekofyear, date_format, asc, udf
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.types import IntegerType
import datetime
from pyspark.sql.functions import col, to_timestamp, month, year
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler


from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import to_date, lit

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator



# Load Data

In [0]:
# Load the data in
# Data comes from https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark#azure-databricks

# Create spark session
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = "r"

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
Register the DataFrame as a SQL temporary view: source
Displaying top 10 rows: 


vendorID,tpepPickupDateTime,tpepDropoffDateTime,passengerCount,tripDistance,puLocationId,doLocationId,startLon,startLat,endLon,endLat,rateCodeId,storeAndFwdFlag,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,totalAmount,puYear,puMonth
CMT,2012-02-29T23:53:14Z,2012-03-01T00:00:43Z,1,2.1,null,null,-73.980494,40.730601,-73.983532,40.752311,1,N,CSH,7.3,0.5,0.5,null,0.0,0.0,8.3,2012,3
VTS,2012-03-17T08:01:00Z,2012-03-17T08:15:00Z,1,11.06,null,null,-73.986067,40.699862,-73.814838,40.737052,1,null,CRD,24.5,0.0,0.5,null,4.9,0.0,29.9,2012,3
CMT,2012-02-29T23:58:51Z,2012-03-01T00:15:48Z,1,3.4,null,null,-73.968967,40.754359,-73.957048,40.743289,1,N,CRD,12.5,0.5,0.5,null,1.5,0.0,15.0,2012,3
CMT,2012-03-01T19:24:16Z,2012-03-01T19:31:22Z,1,1.3,null,null,-73.99374,40.75307,-74.005428,40.741118,1,N,CRD,6.1,1.0,0.5,null,0.0,0.0,7.6,2012,3
CMT,2012-02-29T23:46:32Z,2012-03-01T00:05:18Z,3,2.0,null,null,-73.973723,40.752323,-73.948275,40.769413,1,N,CSH,11.7,0.5,0.5,null,0.0,0.0,12.7,2012,3
VTS,2012-03-07T15:17:00Z,2012-03-07T15:26:00Z,5,1.87,null,null,-73.988237,40.75929,-73.97114,40.78275,1,null,CSH,7.7,0.0,0.5,null,0.0,0.0,8.2,2012,3
CMT,2012-02-29T23:41:58Z,2012-03-01T00:02:29Z,1,12.4,null,null,-73.954536,40.727742,-73.768994,40.760246,1,N,CSH,28.5,0.5,0.5,null,0.0,0.0,29.5,2012,3
VTS,2012-03-18T15:21:00Z,2012-03-18T15:32:00Z,6,2.51,null,null,-74.001705,40.732345,-73.974888,40.750835,1,null,CSH,8.9,0.0,0.5,null,0.0,0.0,9.4,2012,3
CMT,2012-02-29T23:47:08Z,2012-03-01T00:06:42Z,4,6.3,null,null,-73.992319,40.724503,-73.923589,40.76113,1,N,CRD,16.5,0.5,0.5,null,4.37,0.0,21.87,2012,3
VTS,2012-03-13T22:26:00Z,2012-03-13T22:37:00Z,1,1.34,null,null,-74.009907,40.706292,-74.000512,40.71733,1,null,CSH,7.3,0.5,0.5,null,0.0,0.0,8.3,2012,3


In [0]:
# Load in taxi-zone-lookup.csv file because we want to focus on Manhattan data since 90% of the pickups come from there. This means we also minimize the pickup location zones from 265 to 69 pick up zones.  
# First, loaded in the taxi-zone-lookup.csv to the workspace where this notebook is
# file_path = f"/Workspace/Users/{user}/taxi-zone-lookup.csv" 
# taxi_zone = pd.read_csv(file_path)
# taxi_zone

# Filter the pick up zone df to get only Manhattan LocationIDs
# manhattan_df = taxi_zone[taxi_zone['Borough'] == 'Manhattan']
# Get a list of location IDs in Manhattan
# manhattan_location_ids = manhattan_df['LocationID'].tolist()
# manhattan_location_ids

# Truncate Data

In [0]:
# List of puLocationIds to filter for Manhattan rides
puLocationIds_manhattan = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 104, 105, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]
# len(puLocationIds_manhattan) # double check 69 zones. Yes

# Filter for only 2017-2018 data in Manhattan 
filter_conditions = ((col("puYear") >= 2017) & (col("puYear") <= 2018)) & (col("puLocationId").isin(puLocationIds_manhattan))

df_2017_2018_manhattan = df.filter(filter_conditions)

# Data Cleaning

In [0]:
# Drop startLon, startLat, endLon, and endLat columns. They are null & not needed for this project scope
columns_to_drop = ['startLon','startLat', 'endLon', 'endLat']
df_2017_2018_manhattan = df_2017_2018_manhattan.drop(*columns_to_drop)

# Transform non-numeric columns to numeric data types
df_2017_2018_manhattan = df_2017_2018_manhattan \
    .withColumn("vendorID", col("vendorID").cast("int")) \
    .withColumn("puLocationId", col("puLocationId").cast("int")) \
    .withColumn("doLocationId", col("doLocationId").cast("int")) \
    .withColumn("paymentType", col("paymentType").cast("int")) \
    .withColumn("storeAndFwdFlag", when(col("storeAndFwdFlag") == 'Y', 1).otherwise(0).cast("int")) \
    .withColumn("improvementSurcharge", col("improvementSurcharge").cast("double"))

# Remove tripDistance less than 0 and greater than 12. 
# Remove under 1 min rides and greater than 45 min rides.
# Remove rides with no passenegers and more than 5 passengers.
# Remove speeds less than 4 mph and greater than 30 mph.
# Remove fareAmount less than $4 and greater than $52.

# Calculate ride time in minutes (ride_duration)
df_2017_2018_manhattan = df_2017_2018_manhattan.withColumn('ride_duration', # in minutes
                                       (expr('unix_timestamp(tpepDropoffDateTime) - unix_timestamp(tpepPickupDateTime)') / 60).cast('double'))

# Calculate speed, remember this is NOT per hour, it is per minute so need to multiply by 60
df_2017_2018_manhattan = df_2017_2018_manhattan.withColumn('speed', (col('tripDistance') / col('ride_duration')) * 60) # convert minutes to hours (multiply by 60)

# Remove rows
clean_df = df_2017_2018_manhattan[(df_2017_2018_manhattan['tripDistance'] >= 0) & (df_2017_2018_manhattan['tripDistance'] <= 12) & (df_2017_2018_manhattan['ride_duration'] >= 1) & (df_2017_2018_manhattan['ride_duration'] <= 45) & (df_2017_2018_manhattan['passengerCount'] >= 1) & (df_2017_2018_manhattan['passengerCount'] <= 5) & (df_2017_2018_manhattan['speed'] >= 4) & (df_2017_2018_manhattan['speed'] <= 30) & (df_2017_2018_manhattan['fareAmount'] >= 4) & (df_2017_2018_manhattan['fareAmount'] <= 52)]

# Drop columns not needed
# List of columns to drop not useful for this project scope
columns_to_drop = ["vendorID","tpepDropoffDateTime","passengerCount","tripDistance", "doLocationId", "rateCodeId", "storeAndFwdFlag", "paymentType", "fareAmount", "extra", "mtaTax", "improvementSurcharge", "tipAmount", "tollsAmount", "totalAmount", "ride_duration","speed"]

clean_df = clean_df.drop(*columns_to_drop)

# Derive Features

In [0]:
# Create indication if weekday/weekend as this could affect the number of taxis needed.
# Create indication if holiday (1) or not (0).
# Create hour of day, day of week, and week number.
# Create demand column that counts number of taxis per day and hour for EACH pickup zone.

# Indication if weekday/weekend
clean_df = clean_df.withColumn('is_weekend', 
                                         when((dayofweek(clean_df['tpepPickupDateTime']) == 1) | 
                                              (dayofweek(clean_df['tpepPickupDateTime']) == 7), 1).otherwise(0))

# Indication if holiday
# A list of holidays in `holiday_dates`: New Years, MLK day, Presidents day, Memorial Day, 4th of July, Labor Day, Thanksgiving, and Christmas
holiday_dates = ['2017-01-01', '2018-01-01', '2017-01-16', '2018-01-15','2017-02-20','2018-02-19','2017-05-29', '2018-05-28','2017-07-04', '2018-07-04','2017-09-4','2018-09-03','2017-11-23', '2018-11-22', '2017-12-25', '2018-12-25'
]  
# 1 if holiday, 0 if not
clean_df = clean_df.withColumn('is_holiday', when(date_format(clean_df['tpepPickupDateTime'], 'yyyy-MM-dd').isin(holiday_dates), 1).otherwise(0))

# Indicate hour of day, day of week, and week number of year
clean_df = clean_df.withColumn("hour_of_day", F.hour("tpepPickupDateTime"))
clean_df = clean_df.withColumn("day_of_week", F.dayofweek("tpepPickupDateTime"))
# clean_df = clean_df.withColumn('week_number', weekofyear(clean_df['tpepPickupDateTime']))

# weekofyear() function was not calculating week number correctly so created our own function
# Define a UDF to calculate the week number
def custom_week_number(date_time):
    # Get the year of the date
    year = date_time.year
    # Get the first day of the year
    first_day_of_year = datetime.datetime(year, 1, 1)
    # Calculate the day of the week for January 1st
    jan_1_day_of_week = first_day_of_year.isoweekday()
    # Get the current date's day of the week
    current_day_of_week = date_time.isoweekday()
    # Calculate the week number
    if jan_1_day_of_week == 7:  # If January 1st is Sunday
        week_number = (date_time - first_day_of_year).days // 7 + 1
    else:
        week_number = (date_time - first_day_of_year).days // 7 + 1
        if week_number == 0:
            week_number = 52
    # Adjust for the case where December 31st falls on a Sunday or calculated as week number 53
    if date_time.month == 12 and date_time.day == 31:
        if current_day_of_week == 7 or week_number == 53:
            week_number = 52
    return week_number

# Register the UDF
week_number_udf = udf(custom_week_number, IntegerType())

# Apply the UDF to calculate the week number
clean_df = clean_df.withColumn('week_number', week_number_udf(clean_df['tpepPickupDateTime']))

# Now, calculate demand of taxis for each hour of day by zone
# Convert tpepPickupDateTime to a timestamp data type
clean_df = clean_df.withColumn("tpepPickupDateTime", F.to_timestamp("tpepPickupDateTime"))

# Aggregate the df by date, hour, and puLocationId
# Get the data in desired format
aggregated_df = clean_df.groupby(
    F.concat(
        F.date_format("tpepPickupDateTime", "yyyy-MM-dd"),
        F.lit(" "),
        F.format_string("%02d", F.col("hour_of_day"))  # Ensure hour is zero-padded
    ).alias("Date_Hour"),
    "puLocationId"
).agg(
    F.first("day_of_week").alias("day_of_week"),
    F.first("week_number").alias("week_number"),
    F.first("is_weekend").alias("is_weekend"),
    F.first("is_holiday").alias("is_holiday"),
    F.count("*").alias("demand")
)

# Convert the Date_Hour column to timestamp
aggregated_df = aggregated_df.withColumn("Date_Hour", F.to_timestamp("Date_Hour", "yyyy-MM-dd HH"))

# Order the df by Date_Hour in ascending order
aggregated_df = aggregated_df.orderBy(asc("Date_Hour"))

# aggregated_df.show()

# Initial Time-Series Graph

# Linear Model

## Initial Model

## Fine-tune model

In [0]:
# Convert 'Date_Hour' to date type
aggregated_df = aggregated_df.withColumn("Date", to_date("Date_Hour"))

# Split the data into training and testing sets based on the date
train_data = aggregated_df.filter((col('Date') >= lit('2017-01-01')) & (col('Date') <= lit('2018-05-31')))
test_data = aggregated_df.filter((col('Date') >= lit('2018-06-01')) & (col('Date') <= lit('2018-12-31')))

# Define the stages for the pipeline
string_indexer = StringIndexer(inputCol="puLocationId", outputCol="puLocationId_Indexed", handleInvalid="skip")
encoder = OneHotEncoder(inputCols=["puLocationId_Indexed"], outputCols=["puLocationId_OHE"])
assembler = VectorAssembler(inputCols=["puLocationId_OHE", "day_of_week", "week_number", "is_weekend", "is_holiday"], outputCol="features")
# Add regularization to the model
lr = LinearRegression(featuresCol="features", labelCol="demand", elasticNetParam=0.8, regParam=0.1)

# Create the pipeline with the new regularized linear regression
pipeline = Pipeline(stages=[string_indexer, encoder, assembler, lr])

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  # 0.0 = L2, 1.0 = L1
             .addGrid(lr.regParam, [0.1, 0.01, 0.001])
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=pipeline, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=RegressionEvaluator(labelCol="demand"),
                    numFolds=3)

# Run cross validations. This step can take some time to compute.
cvModel = cv.fit(train_data)

# Look at the model coefficients and intercept
bestModel = cvModel.bestModel.stages[-1]
print(f"Coefficients: {bestModel.coefficients}")
print(f"Intercept: {bestModel.intercept}")

# Evaluate best model on test data

Coefficients: [238.426160063352,106.55735860893428,201.491767555364,91.67937709709865,138.71006049308804,56.60703785537175,29.413271085132916,7.21237083188831,59.26370286259451,128.19654798434053,205.78521877804621,-5.3890808904700815,33.348882651236345,-33.982129555074074,289.34349135542345,246.9365115109923,164.6667961669405,133.68981566822185,217.50030787829579,249.10698945806973,78.6038526101135,237.366379575261,70.70500888412437,282.914030913867,323.5354443608307,86.43412942346752,161.75532714397707,29.75053307236997,69.08896186194691,214.99830204386015,-28.61178158972116,132.44745428180735,-92.15247301418086,-57.668807772686435,75.00841046217161,-75.40219742330038,-92.42938019938744,-4.245808375381753,6.657587681664653,-65.63999841027601,-49.550942508159885,-39.36918561781746,-100.03925256612378,-8.417078620063608,-105.89857176730409,0.9136261598478735,27.156059248446976,-115.94266016787492,-121.07201771018923,-90.45627816627687,-68.6864724187703,-113.25423842361279,-28.613995350

In [0]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="demand", metricName="rmse")

# Get the best regularization parameters
bestRegParam = bestModel._java_obj.getRegParam()
bestElasticNetParam = bestModel._java_obj.getElasticNetParam()
print(f"Best regParam: {bestRegParam}")
print(f"Best elasticNetParam: {bestElasticNetParam}")

# Evaluate the best model on the test data
predictions = cvModel.transform(test_data)
rmse = evaluator.evaluate(predictions)
print(f"Test RMSE: {rmse}")

Best regParam: 0.001
Best elasticNetParam: 0.0
Test RMSE: 118.37501501341444
